In [ ]:
import pandas as pd
import numpy as np
import joblib
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler
from sklearn.linear_model import LinearRegression
from xgboost import XGBRegressor
from sklearn.metrics import r2_score

# ✅ 데이터 불러오기
df = pd.read_csv("data/student-por.csv")

# ✅ `G1`, `G2` 변환 (평균 성적, 비율 계산)
df["G_avg"] = df["G1"] + df["G2"]
df["G_ratio"] = df["G1"] / (df["G2"] + 1)  

# ✅ **6개월 기준 누적 공부 시간과 자유 시간 계산**
df["studytime_per_6months"] = df["studytime"] * 26  
df["freetime_per_6months"] = df["freetime"] * 26  
df["total_hours"] = df["studytime_per_6months"] + df["freetime_per_6months"]

# ✅ **공부 시간이 많을수록 성적 상승 효과 증가 (꾸준히 증가)**
df["studytime_boost"] = np.log1p(df["studytime_per_6months"]) * 5  # 🎯 로그 변환으로 안정성 증가

# ✅ **자유 시간이 많을수록 성적 하락 패널티 적용 (완만한 감소)**
df["freetime_penalty"] = np.sqrt(df["freetime_per_6months"]) * 1.5  

# ✅ **학습 시간 비율 조정 (꾸준히 증가)**
df["studytime_ratio"] = (df["studytime_per_6months"] / (df["total_hours"] + 1)) ** 1.3  

# ✅ 학습 데이터 구성
X = df[['studytime_per_6months', 'freetime_per_6months', 'G_avg', 'G_ratio', 
        'total_hours', 'studytime_boost', 'freetime_penalty', 'studytime_ratio']]
y = df[['G3']]

# ✅ MinMaxScaler 적용
scaler_X = MinMaxScaler()
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

X_train_scaled = scaler_X.fit_transform(X_train)
X_test_scaled = scaler_X.transform(X_test)

# ✅ Linear Regression 학습
regressor_lr = LinearRegression()
regressor_lr.fit(X_train_scaled, y_train)

# ✅ XGBoost 학습
regressor_xgb = XGBRegressor(
    n_estimators=500,  
    learning_rate=0.05,  # 학습 속도 조절
    max_depth=6,  # 트리 깊이 증가
    subsample=0.9,  
    colsample_bytree=0.9,  
    random_state=42
)
regressor_xgb.fit(X_train_scaled, y_train.values.ravel())

# ✅ 모델 평가
y_pred_test_lr = regressor_lr.predict(X_test_scaled)
y_pred_test_xgb = regressor_xgb.predict(X_test_scaled)
test_score_lr = r2_score(y_test, y_pred_test_lr) * 100
test_score_xgb = r2_score(y_test, y_pred_test_xgb) * 100
print(test_score_lr)
print(test_score_xgb)
# ✅ 최적 모델 선택
best_model = regressor_xgb if test_score_xgb > test_score_lr else regressor_lr
joblib.dump(best_model, "best_model.pkl")
joblib.dump(scaler_X, "scaler_X.pkl")

print(f"✅ 최적 모델 저장 완료! (`best_model.pkl`)")


83.0240568401325
75.8800745010376
✅ 최적 모델 저장 완료! (`best_model.pkl`)


In [ ]:
import pandas as pd
import numpy as np
import joblib
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, classification_report

# ✅ 데이터 불러오기
df = pd.read_csv("data/student-por.csv")

# ✅ G3을 예측하는 별도의 모델이 존재하므로, 해당 모델의 출력을 가져와 사용한다고 가정

# ✅ 장학금 가능 여부 설정 (예: G3 >= 15이면 장학금 가능, 아니면 불가능)
df['scholarship'] = (df['G3'] >= 15).astype(int)

# ✅ 사용 변수 선택 (⚠️ G3을 입력값으로 사용)
X = df[['G3']]  # 장학금 모델에서는 G3을 입력값으로 사용
y = df['scholarship']

# ✅ 데이터 정규화
scaler = MinMaxScaler()
X_scaled = scaler.fit_transform(X)

# ✅ 데이터 분할
X_train, X_test, y_train, y_test = train_test_split(X_scaled, y, test_size=0.2, random_state=42)

# ✅ 머신러닝 모델 학습 (랜덤 포레스트 분류기 사용)
model = RandomForestClassifier(n_estimators=200, random_state=42)
model.fit(X_train, y_train)

# ✅ 모델 평가
y_pred = model.predict(X_test)
accuracy = accuracy_score(y_test, y_pred)
print(f"✅ 모델 정확도: {accuracy * 100:.2f}%")
print(classification_report(y_test, y_pred))

# ✅ 모델 및 스케일러 저장
joblib.dump(model, "scholarship_model.pkl")
joblib.dump(scaler, "scholarship_scaler.pkl")

print("✅ 모델 및 변환기 저장 완료!")
